In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import sys
import warnings

import pandas as pd

from swmm_api import SwmmInput, CONFIG
from swmm_api.input_file.macros import *

from swmm_model_simplification.simplifying_model import aggregate_model

In [3]:
CONFIG['exe_path'] = "/Users/markus/.bin/runswmm"

In [4]:
inp = SwmmInput('model_bellinge.inp')
set_crs(inp, 'EPSG:25832')

In [5]:
print_summary(inp)

ROUTING_STEP: 5
NODES: 1_020
   JUNCTIONS: 995
   STORAGE: 16
   OUTFALLS: 9
LINKS: 1_044
   CONDUITS: 1015
   WEIRS: 13
   OUTLETS: 7
   ORIFICES: 3
SUBCATCHMENTS: 713


In [6]:
# from swmm_api.input_file.macros.plotting_map import plot_map
# plot_map(inp)

In [7]:
# from swmm_api.input_file.macros.plotting_map_plotly import init_empty_map_plot, add_link_map, add_node_map, add_subcatchment_map
# fig = init_empty_map_plot()
# add_subcatchment_map(fig, inp)
# add_link_map(fig, inp)
# add_node_map(fig, inp)
# fig.layout.update(height=500, width=1000)
# fig.layout.autosize = True
# fig.write_html('model.html')
# fig

In [8]:
from swmm_api.input_file.macros.plotting_map_leaflet import plot_map
m = plot_map(inp)
m.save('map_interactive_leaflet.html')

In [6]:
logger = logging.getLogger('simplify bellinge')
logger.setLevel(logging.ERROR)

# logging.basicConfig(filename=os.path.join(PATH_CHANGES_LOG, 'flows.log'))

fmt = logging.Formatter('[%(asctime)s] %(message)s')

# log to the console/terminal
# console_handler = logging.StreamHandler(sys.stderr)
# console_handler.setFormatter(fmt)
# logger.addHandler(console_handler)

# log to a file
file_handler = logging.FileHandler('log_simplify_bellinge.txt', mode='w', encoding='utf-8')
file_handler.setFormatter(fmt)
logger.addHandler(file_handler)

warnings.showwarning = lambda message, category, filename, lineno, file=None, line=None: logger.warning(str(message))

In [7]:
help(aggregate_model)

Help on function aggregate_model in module swmm_model_simplification.simplifying_model:

aggregate_model(inp_hd, area_min=5, length_max=400, label=None, version=None, skip_optimisation=False, optimize_volume=True, optimize_flow_full=True, optimize_flow_full_ratio=True, write_sc_transformation_as_tag=False, logger_func=None)
    Simplifies SWMM model using the set rainfall. Auto-calibrated based on flow.

    Tested on small rainfall event with no flooding.

    Preparation:
        - set rain for raingauges TIMESERIES, RAINGAUGES, SUBCATCHMENTS
        - set times for rain in OPTIONS
        - set report start for start time of calibration time range for the NSE in OPTIONS
        - set simulation options - routing step, ...

    Args:
        inp_hd (SwmmInput): SWMM input data.
        area_min (int | float):
        length_max (int | float):
        label (str):
        version (int):
        skip_optimisation (bool):
        write_sc_transformation_as_tag (bool):
        logger_fun

In [8]:
# set rain for raingauges TIMESERIES, RAINGAUGES, SUBCATCHMENTS
# set times for rain in OPTIONS
# set simulation options - routing step, ...
inp.OPTIONS.set_routing_step(5)
inp.OPTIONS.set_variable_step(0.75)
inp.OPTIONS.set_lengthening_step(1)
inp.OPTIONS.set_minimum_step(0.1)
inp.OPTIONS.set_max_trials(24)
inp.OPTIONS.set_threads(1)

from swmm_api.input_file.sections import RainGage, TimeseriesData
ts = TimeseriesData('block_rain', [(0, 0), (1, 20)])
inp.add_obj(ts)

rg = RainGage(name='RG', form=RainGage.FORMATS.VOLUME, interval='1:00',
                     SCF=1, source=RainGage.SOURCES.TIMESERIES, timeseries=ts.name)
inp.add_obj(rg)

for sc in inp.SUBCATCHMENTS:
    inp.SUBCATCHMENTS[sc].rain_gage = rg.name

inp.OPTIONS.set_start(pd.Timestamp.today().replace(minute=0, second=0, hour=0, microsecond=0, nanosecond=0))
inp.OPTIONS.set_report_start(inp.OPTIONS.get_start() + pd.Timedelta(hours=1))
inp.OPTIONS.set_simulation_duration(pd.Timedelta(hours=2))

In [12]:
from swmm_api.run_swmm.run_temporary import swmm5_run_temporary
from swmm_model_simplification.helpers.config import CONFIG
with swmm5_run_temporary(inp.copy(), run=CONFIG["SWMM_runner"]) as res:
    out = res.out
    rpt = res.rpt

In [13]:
rpt.flow_routing_continuity

{'Dry Weather Inflow': {'Volume_hectare-m': 0.006, 'Volume_10^6 ltr': 0.063},
 'Wet Weather Inflow': {'Volume_hectare-m': 1.527, 'Volume_10^6 ltr': 15.266},
 'Groundwater Inflow': {'Volume_hectare-m': 0.0, 'Volume_10^6 ltr': 0.0},
 'RDII Inflow': {'Volume_hectare-m': 0.0, 'Volume_10^6 ltr': 0.0},
 'External Inflow': {'Volume_hectare-m': 0.0, 'Volume_10^6 ltr': 0.0},
 'External Outflow': {'Volume_hectare-m': 0.542, 'Volume_10^6 ltr': 5.425},
 'Flooding Loss': {'Volume_hectare-m': 0.07, 'Volume_10^6 ltr': 0.697},
 'Evaporation Loss': {'Volume_hectare-m': 0.0, 'Volume_10^6 ltr': 0.0},
 'Exfiltration Loss': {'Volume_hectare-m': 0.0, 'Volume_10^6 ltr': 0.0},
 'Initial Stored Volume': {'Volume_hectare-m': 0.0, 'Volume_10^6 ltr': 0.0},
 'Final Stored Volume': {'Volume_hectare-m': 0.859, 'Volume_10^6 ltr': 8.59},
 'Continuity Error (%)': 4.028}

In [9]:
inp_simple = aggregate_model(inp, logging_func=None)

simplify model - iter over all nodes:   0%|          | 0/1020 [00:00<?, ?it/s]

In [10]:
print_summary(inp_simple)

ROUTING_STEP: 5
NODES: 133
   JUNCTIONS: 108
   STORAGE: 16
   OUTFALLS: 9
LINKS: 156
   CONDUITS: 127
   WEIRS: 13
   OUTLETS: 7
   ORIFICES: 3
SUBCATCHMENTS: 66


In [11]:
from swmm_api.input_file.macros.plotting_map_leaflet import plot_map
set_crs(inp_simple, 'EPSG:25832')
m = plot_map(inp_simple)
# m.save('map_interactive_leaflet.html')
m